In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup, set_seed
from peft import (
get_peft_model,
PeftType,
PromptTuningConfig
)
import argparse
import os
import torch
from torch.optim import AdamW
from tqdm import tqdm

2025-06-14 13:30:35.014629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749907835.235779      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749907835.296657      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
task = 'mrpc'
datasets = load_dataset('glue', task)
metric = evaluate.load('glue', task)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
datasets['train'][0], datasets['train'][1]

({'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  'label': 1,
  'idx': 0},
 {'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  'label': 0,
  'idx': 1})

In [6]:
datasets['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [7]:
datasets['train'].features['label'].names

['not_equivalent', 'equivalent']

In [8]:
model_name = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
tokenizer

RobertaTokenizerFast(name_or_path='roberta-large', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [10]:
list(tokenizer.vocab.keys())[:10], list(tokenizer.vocab.values())[:10]

(['ĠAtmospheric',
  'Much',
  'ĠRoh',
  'Wr',
  'Ġ8000',
  'Ġconfiguration',
  'Ġimprint',
  'ĠShall',
  'otent',
  'wcs'],
 [30120, 34443, 11854, 44142, 38433, 20393, 31544, 39709, 41098, 49211])

In [11]:
tokenizer('This is a sample text')

{'input_ids': [0, 713, 16, 10, 7728, 2788, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer('This is a sample text').tokens()

['<s>', 'This', 'Ġis', 'Ġa', 'Ġsample', 'Ġtext', '</s>']

In [13]:
def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation = True, max_length = 512)

In [14]:
tokenized_datasets = datasets.map(
    tokenize_function,
    batched = True,
    batch_size = 256,
    remove_columns = ['idx', 'sentence1', 'sentence2']
)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [16]:
tokenized_datasets = tokenized_datasets.rename_columns({'label': 'labels'})

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [18]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding = 'longest', return_tensors = 'pt')

In [19]:
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle = True, collate_fn = collate_fn, batch_size = 32)
eval_dataloader = DataLoader(tokenized_datasets['validation'], shuffle = False, collate_fn = collate_fn, batch_size = 32)

In [20]:
model_name = 'roberta-large'
batch_size = 32
peft_type = PeftType.PROMPT_TUNING

In [21]:
peft_config = PromptTuningConfig(task_type = 'SEQ_CLS', num_virtual_tokens = 10)

In [29]:
lr = 1e-3
num_epochs = 7
device = 'cuda'

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict = True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,061,890 || all params: 356,423,684 || trainable%: 0.2979


In [24]:
model

PeftModelForSequenceClassification(
  (base_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): Robe

In [25]:
optimizer = AdamW(params = model.parameters(), lr = lr)

In [26]:
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer = optimizer,
    num_warmup_steps = 0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps = (len(train_dataloader) * num_epochs)
)

In [30]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=1)
        predictions, references = predictions, batch['labels']
        metric.add_batch(
            predictions = predictions,
            references = references
        )

    eval_metric = metric.compute()
    print(f'epoch {epoch}', eval_metric)

100%|██████████| 13/13 [00:06<00:00,  1.86it/s]


epoch 0 {'accuracy': 0.7058823529411765, 'f1': 0.8208955223880596}


100%|██████████| 13/13 [00:06<00:00,  1.88it/s]


epoch 1 {'accuracy': 0.7058823529411765, 'f1': 0.8198198198198199}


100%|██████████| 13/13 [00:06<00:00,  1.90it/s]


...

epoch 4 {'accuracy': 0.7058823529411765, 'f1': 0.8198198198198199}


100%|██████████| 13/13 [00:06<00:00,  1.91it/s]


epoch 5 {'accuracy': 0.7058823529411765, 'f1': 0.8198198198198199}


100%|██████████| 13/13 [00:06<00:00,  1.90it/s]

epoch 6 {'accuracy': 0.7058823529411765, 'f1': 0.8198198198198199}


In [34]:
model.to(device)
model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=1)
    predictions, references = predictions, batch['labels']
    metric.add_batch(
        predictions = predictions,
        references = references
    )
    eval_metric = metric.compute()
print(eval_metric)

100%|██████████| 13/13 [00:06<00:00,  2.01it/s]

{'accuracy': 0.8333333333333334, 'f1': 0.9047619047619047}


In [35]:
test_batch = tokenizer(['The cat is sitting on the windowsill.'], ['The feline is perched on the window ledge.'], padding = True, truncation = True,
                      return_tensors = 'pt')

In [36]:
test_batch

{'input_ids': tensor([[    0,   133,  4758,    16,  2828,    15,     5,  6410,  1873,     4,
             2,     2,   133,   856,  7012,    16, 32842,    15,     5,  2931,
         41378,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [37]:
test_batch['input_ids'] = test_batch['input_ids'].to(device)
test_batch['attention_mask'] = test_batch['attention_mask'].to(device)

In [42]:
model.to(device)
with torch.no_grad():
    outputs = model(**test_batch)
probs = torch.softmax(outputs.logits, dim=1)
label_idx = probs.argmax()
print('Two sentences are', datasets['train'].features['label'].names[label_idx])

Two sentences are equivalent
